<a href="https://colab.research.google.com/github/hun9008/ML_TeamProject_24SS/blob/main/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /gdrive


In [ ]:
cd /content/drive/MyDrive

In [ ]:
import os
os.getcwd()
os.chdir('..')

In [ ]:
os.listdir()

In [ ]:
os.chdir('ML_TeamProject_24SS')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
import pickle
#os.chdir('images_gray_jpg')

os.chdir('overripe_train_labeled_2000')
with open('zero_centering.pkl', 'rb') as f:
     overripe = pickle.load(f)
     print('overripe : ', len(overripe))
os.chdir("..")
os.chdir('no_train_labeled_2000')
with open('zero_centering.pkl', 'rb') as f:
     no = pickle.load(f)
     print('no : ', len(no))
os.chdir("..")
os.chdir('mature_train_labeled_2000')
with open('zero_centering.pkl', 'rb') as f:
     mature = pickle.load(f)
     print('mature : ', len(mature))
os.chdir("..")
os.chdir('incipient_train_labeled_2000')
with open('zero_centering.pkl', 'rb') as f:
     incipient = pickle.load(f)
     print('incipient : ', len(incipient))

mature :  2000


In [3]:
# overripe의 모든 사진에 대해 반복
overripe_data_list = []

for filename, data in overripe.items():
    zero_centering_value = data.get('zero_centering')
    overripe_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
overripe_data = np.array(overripe_data_list)

# 확인을 위해 배열의 크기 출력
print("overripe_data shape:", overripe_data.shape)
no_data_list = []

for filename, data in no.items():
    zero_centering_value = data.get('zero_centering')
    no_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
no_data = np.array(no_data_list)

# 확인을 위해 배열의 크기 출력
print("no_data shape:", no_data.shape)

mature_data_list = []

for filename, data in mature.items():
    zero_centering_value = data.get('zero_centering')
    mature_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
mature_data = np.array(mature_data_list)

# 확인을 위해 배열의 크기 출력
print("mature_data shape:", mature_data.shape)

incipient_data_list = []

for filename, data in incipient.items():
    zero_centering_value = data.get('zero_centering')
    incipient_data_list.append(np.array(zero_centering_value))

# 리스트를 NumPy 배열로 변환
incipient_data = np.array(incipient_data_list)

# 확인을 위해 배열의 크기 출력
print("incipient_data shape:", incipient_data.shape)

incipient :  2000


In [4]:
# 데이터를 train/validation/test로 나누기
from sklearn.model_selection import train_test_split
X = np.concatenate((overripe_data, no_data, mature_data, incipient_data), axis=0)
y = np.concatenate((np.zeros(overripe_data.shape[0]), np.ones(no_data.shape[0]),
                    2*np.ones(mature_data.shape[0]), 3*np.ones(incipient_data.shape[0])))

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

mature_train_labeled_value shape: (2000, 224, 224, 3)


In [5]:
y_train_one_hot = to_categorical(y_train)
y_val_one_hot = to_categorical(y_val)
y_test_one_hot = to_categorical(y_test)

incipient_train_labeled_value shape: (2000, 224, 224, 3)


In [18]:

# 모델 설계
input_shape = (224, 224, 3)  # 이미지 크기와 채널 수 지정

# 모델 구성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4개의 클래스에 대한 출력
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train_one_hot, batch_size=32, epochs=30, validation_data=(X_val, y_val_one_hot))

Epoch 1/30
88/88 [==============================] - 7s 36ms/step - loss: 6.3371 - accuracy: 0.4950 - val_loss: 11.7527 - val_accuracy: 0.5017
Epoch 2/30
88/88 [==============================] - 2s 22ms/step - loss: 0.7325 - accuracy: 0.4814 - val_loss: 1.8622 - val_accuracy: 0.4800
Epoch 3/30
88/88 [==============================] - 2s 23ms/step - loss: 0.7077 - accuracy: 0.5018 - val_loss: 0.9426 - val_accuracy: 0.5000
Epoch 4/30
88/88 [==============================] - 2s 23ms/step - loss: 0.7055 - accuracy: 0.5093 - val_loss: 0.7621 - val_accuracy: 0.4867
Epoch 5/30
88/88 [==============================] - 2s 23ms/step - loss: 0.6971 - accuracy: 0.5104 - val_loss: 0.7108 - val_accuracy: 0.4817
Epoch 6/30
88/88 [==============================] - 2s 23ms/step - loss: 0.6923 - accuracy: 0.5096 - val_loss: 0.7077 - val_accuracy: 0.4883
Epoch 7/30
88/88 [==============================] - 2s 23ms/step - loss: 0.6928 - accuracy: 0.5104 - val_loss: 0.6991 - val_accuracy: 0.5017
Epoch 8/30
8

In [20]:
# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test_one_hot)
val_acc = model.evaluate(X_val, y_val_one_hot)

print('Val accuracy:', val_acc)
print('Test accuracy:', test_acc)

19/19 [==============================] - 0s 10ms/step - loss: 0.6914 - accuracy: 0.5017
Val accuracy: [0.6914002299308777, 0.5016666650772095]
Test accuracy: 0.4633333384990692


In [17]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_78 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_79 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 112, 112, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_80 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_81 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 56, 56, 128)       0         
 ng2D)                                                



1.   softmax : 0.5 / 0.5
2.   sigmoid : 0.51 / 0.498

